In [8]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.transforms import Affine2D
import time
from matplotlib.offsetbox import AnchoredText 
import mplcursors 
import os
import json
from gearbox import Gearbox

import itertools
import math

from tqdm import tqdm

## To Move into .py helper file:

variables: z_s, z_p2, z_r2, xs, xp1, xp2, xr2

In [2]:
config = {
  "MODULE": 0.8e-3,
  "MC": 0.8467e-3,
  "MU": 0.21,
  "ALPHA": 20,
  "XS": 0.476,
  "XP1": 0.762,
  "XP2": 0.536,
  "XR1": 2,
  "XR2": 1.21,
  "XS_RANGE": [-0.5, 0.5, 0.25],
  "XP1_RANGE": [-1, 1, 0.25],
  "XP2_RANGE": [-0.5, 0.5, 0.25],
  "XR2_RANGE": [-1.5, 1.5, 0.25],
  "N_PLANETS": 3,
  "SUN_LIMITS": [4, 26],
  "P1_LIMITS": [32, 42],
  "P2_LIMITS": [25, 35],
  "R2_LIMITS": [70,86],
  "STEPS": [2, 1, 1, 1],
  "MIN_RATIO_THRESHOLD": 90,
  "MIN_FORWARD_THRESHOLD": 0.30,
  "MIN_BACKWARD_THRESHOLD": 0.30
}


In [3]:
MODULE = config["MODULE"]
MU = config["MU"]
MC = config.get("MC", MODULE)
ALPHA = np.radians(config.get("ALPHA", 20))
XR1 = config.get("XR1", 0.0)
N_PLANETS = config["N_PLANETS"]
SUN_LIMITS = tuple(config["SUN_LIMITS"])
P1_LIMITS  = tuple(config["P1_LIMITS"])
P2_LIMITS  = tuple(config["P2_LIMITS"])
R2_LIMITS  = tuple(config["R2_LIMITS"])
STEPS = tuple(config["STEPS"])
MIN_RATIO_THRESHOLD = config["MIN_RATIO_THRESHOLD"]
MIN_FORWARD_THRESHOLD = config["MIN_FORWARD_THRESHOLD"]
MIN_BACKWARD_THRESHOLD = config["MIN_BACKWARD_THRESHOLD"]

SLICE_GEAR_RATIO = True 
TARGET_GEAR_RATIO = 100
RATIO_TOLERANCE = 10
DISPLAY_BEST_GEARBOX = False
DISPLAY_ITERATIONS_PLOT = True
RUN_SAMPLE_TRIAL = False
MAX_CANDIDATES = 1000

In [4]:
def generate_options(lower: int, upper: int, step: int = 1):
    return list(range(lower, upper + 1, step))

def generate_range_options(range_vals):
    low, high, step = range_vals
    opts = []
    current = low
    while current <= high + 1e-9:
        opts.append(round(current, 5))
        current += step
    return opts

def generate_all_options(sun_limits: tuple, p1_limits: tuple, p2_limits: tuple, r2_limits: tuple, 
                         steps: tuple = (1,1,1,1)):
    step_sun, step_p1, step_p2, step_r2 = steps
    return {
        "sun": generate_options(sun_limits[0], sun_limits[1], step_sun),
        "p1": generate_options(p1_limits[0], p1_limits[1], step_p1),
        "p2": generate_options(p2_limits[0], p2_limits[1], step_p2),
        "r2": generate_options(r2_limits[0], r2_limits[1], step_r2)
    }

In [5]:
options = generate_all_options(SUN_LIMITS, P1_LIMITS, P2_LIMITS, R2_LIMITS, STEPS)
sun_options = options["sun"]
p1_options  = options["p1"]
p2_options  = options["p2"]
r2_options  = options["r2"]
xs_options = generate_range_options(config["XS_RANGE"])
xp1_options = generate_range_options(config["XP1_RANGE"])
xp2_options = generate_range_options(config["XP2_RANGE"])
xr2_options = generate_range_options(config["XR2_RANGE"])

# Main

In [12]:
#generate all possible subsets
num_per_param = 4

all_params = [sun_options, p2_options, r2_options, xs_options, xp1_options, xp2_options, xr2_options]

ss_vals = []

for param in all_params:
    ss_vals.append(list(itertools.combinations(param, num_per_param)))

test_len = total_combinations = math.prod(len(subsets) for subsets in ss_vals)

# print(f'Total designs to test: {len(list(itertools.product(*ss_vals)))}')
print(f'Total designs to test: {test_len}')

Total designs to test: 875613989250000


In [22]:
import sys

start_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from utils import score_vals

In [24]:
# score the subsets, keep the highest score
high_score = 0
best_vals = []
counter = 0
for combination_of_vals in itertools.product(*ss_vals):
    score = score_vals(list(combination_of_vals))
    if score > high_score:
        best_vals = combination_of_vals
        high_score = score
    counter+=1
    if counter > 1000:
        break

c:\Users\campb\Python_Code\Mod_DOE\utils.py:54: RuntimeWarning: divide by zero encountered in divide
  ratios = 1 / gr_s
c:\Users\campb\Python_Code\Mod_DOE\utils.py:80: RuntimeWarning: divide by zero encountered in divide
  ratios = 1 / gr_s


In [19]:
best_vals

((4, 6, 8, 10),
 (25, 26, 27, 28),
 (70, 71, 72, 73),
 (-0.5, -0.25, 0.0, 0.25),
 (-1, -0.75, -0.5, -0.25),
 (-0.5, -0.25, 0.0, 0.25),
 (0.0, 1.0, 1.25, 1.5))

In [20]:
high_score

2584